In [3]:
import numpy as np
import sympy as sp

from system import *

### Initialize variables

In [4]:
a, b, c = sp.symbols('a b c')

# Computation of some classes
U2 = sp.factor(System({ a, b, c }, [], [ a*c ]).compute_class())

# The classes of the orbit spaces
S = q - 1 # isomorphic to C*
J = q - 1 # isomorphic to C*
M = (q - 1)**2 # isomorphic to C* x C*

# Subspaces of U
SS = S # because they are isomorphic
JJ = J * (q - 1) # trivial fibration JJ = J x C*
MM = sp.factor(U2 - SS - JJ) # this is the easy way (however, System({ a, b, c }, [], [ a - c, a*c ]).compute_class() would also work)

# Classes of orbits and stabilizers of generic S, J or M elements
orbit_generic_S = 1 # singleton orbit
orbit_generic_J = q - 1 # orbit is C*
orbit_generic_M = q # matrices of given a and c (b is free)

stab_generic_S = U2 # whole group stabilizes scalar matrices
stab_generic_J = (q - 1) * q # entries on diagonal must be the same
stab_generic_M = (q - 1)**2 # diagonal matrices

In [5]:
# Some checks
print(sp.expand(orbit_generic_S * stab_generic_S - U2) == 0)
print(sp.expand(orbit_generic_J * stab_generic_J - U2) == 0)
print(sp.expand(orbit_generic_M * stab_generic_M - U2) == 0)

True
True
True


In [6]:
# Used for creating some expressions
x, a1, b1, c1, a2, b2, c2 = sp.symbols('x a1 b1 c1 a2 b2 c2')
j = np.matrix([[ 1, x ], [ 0, 1 ]])
g1 = np.matrix([[ a1, b1 ], [ 0, c1 ]])
g2 = np.matrix([[ a2, b2 ], [ 0, c2 ]])
g1_inv = np.matrix([[ c1, -b1 ], [ 0, a1 ]])
g2_inv = np.matrix([[ c2, -b2 ], [ 0, a2 ]])
det_g1 = a1*c1
det_g2 = a2*c2

In [7]:
# Define indices for the set of generators we consider
T_S = 0
T_J = 1
T_M = 2

### Computing $Z_\pi(L)$

In [8]:
# Computing Z_pi(L)
Z_pi_L = np.matrix(np.zeros((3, 3), dtype = object))

# On T_S:
C = g1 * g2 * g1_inv * g2_inv
Z_pi_L[T_S, T_S] = sp.factor(U2 * System({ a1, b1, c1, a2, b2, c2 }, [ C[0,0] - det_g1 * det_g2, C[0,1], C[1,1] - det_g1 * det_g2 ], [ det_g1, det_g2 ]).compute_class())
Z_pi_L[T_J, T_S] = sp.factor(U2 * U2 * U2 - Z_pi_L[T_S, T_S])
Z_pi_L[T_M, T_S] = 0

# On T_J:
Z_pi_L[T_S, T_J] = Z_pi_L[T_J, T_S] # isomorphic
Z_pi_L[T_J, T_J] = sp.factor(orbit_generic_J * U2 * U2 * U2 - Z_pi_L[T_S, T_J]) # every j [g1, g2] has trace equal to tr(j)
Z_pi_L[T_M, T_J] = 0

# On T_M:
Z_pi_L[T_M, T_M] = orbit_generic_M * U2 * U2 * U2

### Computing $Z_\pi(L_{\mathcal{E}})$

In [9]:
# Computing Z(parabolic)
Z_pi_para_S = np.matrix(np.zeros((3, 3), dtype = object))
Z_pi_para_S[T_S, T_S] = orbit_generic_S * U2
Z_pi_para_S[T_J, T_J] = orbit_generic_J * U2
Z_pi_para_S[T_M, T_M] = orbit_generic_M * U2

Z_pi_para_J = np.matrix(np.zeros((3, 3), dtype = object))
Z_pi_para_J[T_J, T_S] = orbit_generic_S * U2 * orbit_generic_J
Z_pi_para_J[T_S, T_J] = orbit_generic_J * U2
Z_pi_para_J[T_J, T_J] = orbit_generic_J * U2 * (orbit_generic_J - 1)
Z_pi_para_J[T_M, T_M] = orbit_generic_M * U2 * orbit_generic_J

Z_pi_para_M = np.matrix(np.zeros((4, 4), dtype = object))
Z_pi_para_M[T_M, T_S] = orbit_generic_S * U2 * orbit_generic_M
Z_pi_para_M[T_M, T_J] = orbit_generic_J * U2 * orbit_generic_M
Z_pi_para_M[T_M, T_M] = orbit_generic_M * U2 * orbit_generic_M
Z_pi_para_M[T_S, T_M + 1] = orbit_generic_M * U2
Z_pi_para_M[T_J, T_M + 1] = orbit_generic_M * U2 * (orbit_generic_M - 1)

### The map $\eta = \pi_! \pi^*$

In [10]:
# Computing eta:
eta = np.matrix(np.zeros((3, 3), dtype = object))

# All T_S, T_J and T_M are simply points:
eta[T_S, T_S] = orbit_generic_S
eta[T_J, T_J] = orbit_generic_J
eta[T_M, T_M] = orbit_generic_M

# Computing eta_inv:
eta_inv = np.matrix(np.zeros((3, 3), dtype = object))
eta_inv[T_S, T_S] = sp.Integer(1) / orbit_generic_S
eta_inv[T_J, T_J] = sp.Integer(1) / orbit_generic_J
eta_inv[T_M, T_M] = sp.Integer(1) / orbit_generic_M

### Computing $\tilde{Z}$

In [11]:
# Compute Z_tilde_L
Z_tilde_L = Z_pi_L * eta_inv

# Basis for diagonalizing Z_tilde_L
A = np.matrix([[ 1, 1, 0], [ -1, q - 1, 0 ], [ 0, 0, 1 ]])
A_inv = np.matrix([[ (q - 1) / q, -1 / q, 0], [ 1 / q, 1 / q, 0 ], [ 0, 0, 1 ]])

# Look at eigenvalues
sp.factor(A_inv * Z_tilde_L * A)

[[q**3*(q - 1)**4, 0, 0], [0, q**3*(q - 1)**6, 0], [0, 0, q**3*(q - 1)**6]]

In [12]:
# Compute Z_tilde(parabolic)
Z_tilde_para_S = Z_pi_para_S * eta_inv
Z_tilde_para_J = Z_pi_para_J * eta_inv
# ZZ_para_M = ?

In [13]:
# Diagonalize Z_tilde_para_J / (q * (q - 1)**2)
P, D = sp.Matrix(sp.factor(Z_tilde_para_J / (q * (q - 1)**2))).diagonalize()
P

Matrix([
[-1, 1/(q - 1), 0],
[ 1,         1, 0],
[ 0,         0, 1]])

In [14]:
D

Matrix([
[-1,     0,     0],
[ 0, q - 1,     0],
[ 0,     0, q - 1]])

### Computing class of character variety

In [15]:
# Case pure genus
g = sp.Symbol('g')
Chi = q**(2*g - 1) * (q - 1)**(2*g + 1) * ((q - 1)**(2*g - 1) + 1) # from above

Chi_s = (q - 1)**(2*g)
Chi_js = (q - 1)**(2*g) * q**(2*g)
Chi_ms = q * ((q - 1)**(4*g) - (q - 1)**(2*g)) + (q - 1)**(2*g)
Chi_j = Chi_js - Chi_s
Chi_m = Chi_ms - Chi_s
Chi_f = Chi - Chi_s - Chi_j - Chi_m

R = Chi_f / (q * (q - 1)) + Chi_s + Chi_j / (q - 1) + Chi_m / q

# Result will be this
R_g = (q - 1)**(2*g - 1)*(q*(q - 1)**(2*g) - 2*(q - 1)**(2*g) + q**(2*g - 2)*(q - 1)**(2*g) - q**(2*g - 2) + q**(2*g))
print(sp.simplify(R - R_g) == 0)

True


In [16]:
# Case genus and parabolic jordan
g = sp.Symbol('g')
l = sp.Symbol('l')

Chi = q**(2*g - 1) * (q - 1)**(2*g) * ((-1)**l * (q - 1) + (q - 1)**(2*g + l)) # from above

Chi_s = 0
Chi_js = (q - 1)**(2*g) * q**(2*g) * (q - 1)/q * ((q - 1)**(l - 1) + (-1)**l)
Chi_ms = 0
Chi_j = Chi_js - Chi_s
Chi_m = Chi_ms - Chi_s
Chi_f = Chi - Chi_s - Chi_j - Chi_m

R = Chi_f / (q * (q - 1)) + Chi_s + Chi_j / (q - 1) + Chi_m / q

# Result will be this
R_gj = q**(2*g - 2)*(q - 1)**(2*g) * ((-1)**l * q + (q - 1)**l * ((q - 1)**(2*g - 1) + 1))
print(sp.simplify(R - R_gj) == 0)

True


In [17]:
# Case genus and parabolic mixed
g = sp.Symbol('g')
m = sp.Symbol('m')

Chi = q**(2*g + m - 1) * (q - 1)**(4*g) # from above

Chi_s = 0
Chi_js = 0
Chi_ms = q * (q - 1)**(4*g)
Chi_j = Chi_js - Chi_s
Chi_m = Chi_ms - Chi_s
Chi_f = Chi - Chi_s - Chi_j - Chi_m

R = Chi_f / (q * (q - 1)) + Chi_s + Chi_j / (q - 1) + Chi_m / q

# Result will be this
R_gm = (q - 1)**(4*g - 1) * (q - 2 + q**(2*g + m - 2))
print(sp.simplify(R - R_gm) == 0)

True


In [18]:
# Case genus and parabolic jordan + mixed
g = sp.Symbol('g')
l = sp.Symbol('l')
m = sp.Symbol('m')

Chi = q**(2*g + m - 1) * (q - 1)**(4*g + l) # from above

Chi_s = 0
Chi_js = 0
Chi_ms = 0
Chi_j = Chi_js - Chi_s
Chi_m = Chi_ms - Chi_s
Chi_f = Chi - Chi_s - Chi_j - Chi_m

R = Chi_f / (q * (q - 1)) + Chi_s + Chi_j / (q - 1) + Chi_m / q

# Result will be this
R_gjm = q**(2*g + m - 2) * (q - 1)**(4*g + l - 1)
print(sp.simplify(R - R_gjm) == 0)

True
